# Boltzmann Machine

In [1]:
##Downloading the dataset

###ML-100K
'''
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls
'''
###ML-1M
'''
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls
'''

'\n!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"\n!unzip ml-1m.zip\n!ls\n'

## Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [3]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set

In [4]:
# [user, movie, rating, time_stamp]
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [5]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns

In [6]:
# creat list of list with each user with column all the movies with ratings
# [ [1, 2, 0, 1 ....], .....] rating if the movie rated by the user or zero
# |  |              |
# |  Movie_1, ......|
# [User_1___________], .....]
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        # movie ids rated by id_user
        # [data[:,0] == id_users] : get rows only with user == id_users 
        id_movies = data[:, 1] [data[:, 0] == id_users]
        id_ratings = data[:, 2] [data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        # id_movies start with index One
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [7]:
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [9]:
# convert 0 -> -1 : 0 means movie is not rated by the user

training_set[training_set == 0] = -1
# 0 for low ratings (1 or 2 stars)
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
# 1 for higher ratings (3,4,5 stars)
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
# 0 for low ratings (1 or 2 stars)
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
# 1 for higher ratings (3,4,5 stars)
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network

In [10]:
class RBM():

    def __init__(self, nv, nh ):# nv and nh : number of visible and hidden nodes.
        self.W = torch.randn(nh, nv) # init weights(size nh by nv) random with normal distribution
        # Bias for probabilities of hidden nodes given the visible nodes, dimensions batch and bias
        self.a = torch.randn(1, nh)
        #                    |   |_Bias
        #                    Batch
        # Bias for probabilities of visible nodes given the hidden nodes, dimensions batch and bias
        self.b = torch.randn(1, nv)

    def sample_h(self, x):# sampling the hidden node according to probabilities of hidden nodes given visible nodes. (sigmoid activation function), x : values of visible nodes, 'v' in the probabilities of hidden nodes (ph) given visible nodes (v).
        # compute the probability of h given v (probability that hidden neuron equals one given the values of visible neurons)
        wx = torch.mm(x, self.W.t()) # product w, x, t() : get transpose
        activation = wx + self.a.expand_as(wx) # expand_as(wx) : make sure to add bias to each line of minibatchs
        p_h_given_v = torch.sigmoid(activation)# apply the sigmoid function to activations
        # torch.bernoulli(p_h_given_v) : bernoulli sampling
        # Draws binary random numbers (0 or 1) from a Bernoulli distribution
        # if i'th visible node with probability = 0.25, then get random number between 0 and 1 if the random number < 0.25 then the i'th visible node get value 1 else 0
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y): # y : values of hidden nodes
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    # k-step contractat divergence
    # v0 : input vector
    # vk : visible node after k samplings 
    # ph0 : vector of probabilities at the first iteration, hidden node equal one with given values of v0
    # phk : probabilities of the hidden node after k samplings given the values of visible node vk
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

## Training the RBM

In [11]:
nv = len(training_set[0]) # visible nodes
nh = 100 # hidden nodes
batch_size = 100

rbm = RBM(nv, nh)

nb_epoch = 10 # number of epochs
for epoch in range(1, nb_epoch + 1):
    train_loss = 0 # simple difference and absolute value (lost function)
    s = 0.0 # counter for normalize the error

    # batch
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size] # for keep the original ratings of users 
        ph0,_ = rbm.sample_h(v0)

        # k-steps of contrastive divergent (random walk) (gibbs sampling)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # -1 ratings (not rated by user) keep -1
        
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk) # update weights and bias
        # update the train lost
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0])) # calculate simple difference and absolute value (ignore -1 ratings)
        # update the counter
        s += 1.0
    print(f"epoch : {epoch} loss : {train_loss/s}")

epoch : 1 loss : 0.3405577838420868
epoch : 2 loss : 0.23860925436019897
epoch : 3 loss : 0.253926157951355
epoch : 4 loss : 0.24895991384983063
epoch : 5 loss : 0.24922935664653778
epoch : 6 loss : 0.24672900140285492
epoch : 7 loss : 0.24650555849075317
epoch : 8 loss : 0.24651718139648438
epoch : 9 loss : 0.2483144849538803
epoch : 10 loss : 0.24779851734638214


## Testing the RBM

In [12]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2444)
